<a href="https://colab.research.google.com/github/Chanchalg05/LLMS/blob/main/Chat_with_documents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install langchain
!pip install tiktoken
!pip install unstructured[local-inference]

!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.5 MB/s eta 0:00:00


In [22]:

import os
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Pinecone, Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI

In [38]:

os.environ['OPENAI_API_KEY'] ="OPENAI_API_KEY"

In [72]:

from langchain.document_loaders import DirectoryLoader

pdf_loader = DirectoryLoader('/content/', glob="**/*.pdf")
readme_loader = DirectoryLoader('/content/', glob="**/*.json")

In [73]:
loaders = [pdf_loader, readme_loader]

#lets create document
documents = []
for loader in loaders:
    documents.extend(loader.load())

In [74]:
print (f'You have {len(documents)} document(s) in your data')
print (f'There are {len(documents[0].page_content)} characters in your document')

You have 3 document(s) in your data
There are 1711 characters in your document


In [75]:
from langchain.schema.messages import ChatMessageChunk
##Split the text from documents to manage according to token ,limit
text_splitter=CharacterTextSplitter(chunk_size=1000,chunk_overlap=40)
documents=text_splitter.split_documents(documents)
print(len(documents))


41


In [76]:
#embedding and storing it in vectorstore
embeddings=OpenAIEmbeddings()

In [77]:
#using Chroma for storing vectors
from langchain.vectorstores import Chroma


In [14]:
# @title Using  pinecone for storing vectors

%%capture
!pip install pinecone-client

In [78]:

import os
import getpass
PINECONE_API_KEY = getpass.getpass('Pinecone API Key:')


Pinecone API Key:··········


In [79]:
PINECONE_ENV = getpass.getpass('Pinecone Environment:')

Pinecone Environment:··········


In [80]:
import pinecone

##initialize pinecone

pinecone.init(api_key=PINECONE_API_KEY,
              environment=PINECONE_ENV)

index_name="langchain-demo"

vectorstore=Pinecone.from_documents(documents,embeddings,index_name=index_name)


In [81]:
query="who is the authors of Improved Biomedical Entity Recognition via longer context modeling paper"
docs=vectorstore.similarity_search(query)

In [60]:
len(docs)

4

In [61]:
print(docs[0].page_content)

Improved Biomedical Entity Recognition via longer context modeling

Nikolaos Stylianou1, Panagiotis Kosmoliaptsis1,2, and Ioannis Vlahavas1

1 Aristotle University of Thessaloniki, Thessaloniki 54124, Greece 2 General Hospital of Thessaloniki ”George Papanikolaou”, Thessaloniki 57010, Greece {nstylia,kosmoliap,vlahavas}@csd.auth.gr


In [82]:
# @title Now coming to lang chain part where we will chaining the chat history so the chats gets saved in the memory


from langchain.llms import OpenAI

In [83]:
retriever=vectorstore.as_retriever(search_type="similarity",search_kwargs={"k":2})
qa=ConversationalRetrievalChain.from_llm(OpenAI(temperature=0),retriever)

In [84]:
chat_history = []
query="which approach is implemented in the paper?"
result=qa({"question":query,"chat_history":chat_history})
result["answer"]

' The approach implemented in the paper is evaluating the performance of a model on four datasets with four different scopes while modeling sentence-level input and whole abstracts.'

In [85]:
chat_history.append((query,result["answer"]))
chat_history

[('which approach is implemented in the paper?',
  ' The approach implemented in the paper is evaluating the performance of a model on four datasets with four different scopes while modeling sentence-level input and whole abstracts.')]

In [66]:
query="what are the models compared with?"
result=qa({"question":query,"chat_history":chat_history})
result["answer"]

' The models being compared in the paper are Transformer architectures.'

In [69]:
query="can you provide more information about the transformer architecture?"
result=qa({"question":query,"chat_history":chat_history})
result["answer"]

' The paper does not specify which transformer architecture is used.'

In [71]:
query="How many types of series are there?"
result=qa({"question":query,"chat_history":chat_history})
result["answer"]

' Four datasets and four scopes.'

In [ ]:
query="How many types of series are there?"
result=qa({"question":query,"chat_history":chat_history})
result["answer"]

In [ ]:
# @title Create a chatbot




In [98]:
!pip3 install IPython

In [99]:
from IPython import display
import ipywidgets as widgets

In [102]:
chat_history=[]

def on_submit(_):
    query=input_box.value
    input_box.value=""

    if query.lower()=='exit':
      print("Thanks for the chat")
      return


    result=qa({"question":question,"chat_history":chat_history})
    chat_history.append((query,result["answer"]))


    display(widgets.HTML(f'<b>User:</b>{query}'))
    display(widgets.HTML(f'<b><font color="Red">Chatbot:</font></b>{result["answer"]}'))

print("Chat with your data.Type exit to stop")

input_box=widgets.Text(placeholder="Please enyter your question")
input_box.on_submit(on_submit)

display(input_box)

Chat with your data.Type exit to stop


TypeError: ignored